In [1]:
!git clone https://github.com/Lightning-AI/lit-gpt

Cloning into 'lit-gpt'...
remote: Enumerating objects: 2767, done.
remote: Counting objects: 100% (1340/1340), done.
remote: Compressing objects: 100% (579/579), done.
remote: Total 2767 (delta 1124), reused 766 (delta 761), pack-reused 1427
Receiving objects: 100% (2767/2767), 877.54 KiB | 17.91 MiB/s, done.
Resolving deltas: 100% (1845/1845), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd lit-gpt

/content/lit-gpt


In [4]:
!pwd

/content/lit-gpt


In [5]:
!pip install -r requirements.txt

  Cloning https://github.com/Lightning-AI/lightning (to revision master) to /tmp/pip-install-izrzga50/lightning_9168aac614ab43af949306b7483a2a55
  Running command git clone --filter=blob:none --quiet https://github.com/Lightning-AI/lightning /tmp/pip-install-izrzga50/lightning_9168aac614ab43af949306b7483a2a55
  Resolved https://github.com/Lightning-AI/lightning to commit 27cab24ca3f8869a734a6a7bf3e1487337092a38
  Running command git submodule update --init --recursive -q
  Encountered 31 file(s) that should have been pointers, but weren't:
        .notebooks/course_UvA-DL/01-introduction-to-pytorch.ipynb
        .notebooks/course_UvA-DL/02-activation-functions.ipynb
        .notebooks/course_UvA-DL/03-initialization-and-optimization.ipynb
        .notebooks/course_UvA-DL/04-inception-resnet-densenet.ipynb
        .notebooks/course_UvA-DL/05-transformers-and-MH-attention.ipynb
        .notebooks/course_UvA-DL/06-graph-neural-networks.ipynb
        .notebooks/course_UvA-DL/07-deep-energy

In [6]:
!pip install --index-url https://download.pytorch.org/whl/nightly/cu118 --pre 'torch>=2.1.0dev' --pre 'torchaudio>=2.1.0dev'

Looking in indexes: https://download.pytorch.org/whl/nightly/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 774.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 MB 17.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.0.2+cu118
    Uninstalling torchaudio-2.0.2+cu118:
      Successfully uninstalled torchaudio-2.0.2+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.12 requires torch<2.1,>=1.7, but you have torch 2.1.0.dev20230722+cu118 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 2.

In [7]:
# download the model weights
!python scripts/download.py --repo_id tiiuae/falcon-7b

# convert the weights to Lit-Parrot format
!python scripts/convert_hf_checkpoint.py --checkpoint_dir checkpoints/tiiuae/falcon-7b

Fetching 5 files:   0% 0/5 [00:00<?, ?it/s]













































































































































































Fetching 5 files: 100% 5/5 [00:47<00:00,  9.47s/it]
Model config {'org': 'tiiuae', 'name': 'falcon-7b', 'block_size': 2048, 'vocab_size': 50254, 'padding_multiple': 512, 'padded_vocab_size': 65024, 'n_layer': 32, 'n_head': 71, 'n_embd': 4544, 'rotary_percentage': 1.0, 'parallel_residual': True, 'bias': False, 'n_query_groups': 1, 'shared_attention_norm': True, '_norm_class': 'LayerNorm', 'norm_eps': 1e-05, '_mlp_class': 'GptNeoxMLP', 'intermediate_size': 18176, 'condense_ratio': 1}
Processing checkpoints/tiiuae/falcon-7b/pytorch_model-00001-of-00002.bin
Processing checkpoints/tiiuae/falcon-7b/pytorch_model-00002-of-00002.bin


In [13]:
import json
import sys
from pathlib import Path

import requests
import torch
from torch.utils.data import random_split
from tqdm import tqdm
from lit_gpt.tokenizer import Tokenizer

In [14]:
import os

directory = os.getcwd()

print(directory)

/content/lit-gpt


In [15]:
import json

file_path = "/content/drive/MyDrive/counsel_chat_250-tokens_full.json"
processed_data_path = "/content/drive/MyDrive/processed_data.json"

new_data = []

with open(file_path, "r") as file:
    data = json.load(file)

for item in data["train"]:
    for utterance in item["utterances"]:
        if 'history' in utterance and 'candidates' in utterance:
            new_utterance = {
                "instruction": "Pretend you are a psychologist and you need to help your client with his problem.",
                "input": " ".join(utterance.pop("history")),
                "output": " ".join(utterance.pop("candidates"))
            }
            new_data.append(new_utterance)

with open(processed_data_path, 'w') as json_file:
    json.dump(new_data, json_file)

In [13]:
with open(processed_data_path, 'r') as file:
    data = json.load(file)

if len(data) >= 3:
    for i in range(3):
        print("Input: ", data[i]["input"] if data[i]["input"] else "No input data")  # Print the first item from the 'input' list
        print("Output: ", data[i]["output"] if data[i]["output"] else "No output data")  # Print the first item from the 'output' list
        print("---")
else:
    print(f"The data list only contains {len(data)} item(s).")

Input:  can i change my feeling of being worthless to everyone ? i ' m going through some things with my feelings and myself . i barely sleep and i do nothing but think about how i ' m worthless and how i shouldn ' t be here . i ' ve never tried or contemplated suicide . i ' ve always wanted to fix my issues , but i never get around to it . how can i change my feeling of being worthless to everyone ?
Output:  maybe lower your expectations for a bit if you are whole - heartedly committed to moving past the sexual and romantic parts of your relationship and just having a friendship than refraining from all the touching would be a good place to start very often , one person wants to deal with the conflict right away or shortly thereafter and the other person wants to wait " my best guess is that your boyfriend is triggered by some previous relationship , either romantic or in childhood can he do that for you " friend " is a broad category in general , i usually let the client decide when 

In [16]:

DESTINATION_PATH = Path("/content/drive/MyDrive")
CHECKPOINT_DIR = Path("checkpoints/tiiuae/falcon-7b")
TEST_SPLIT_FRACTION = 0.03865
IGNORE_INDEX = -1
MASK_INPUTS = False

In [27]:
def prepare(
    destination_path: Path = DESTINATION_PATH,
    checkpoint_dir: Path = CHECKPOINT_DIR,
    test_split_fraction: float = TEST_SPLIT_FRACTION,
    seed: int = SEED,
    mask_inputs: bool = MASK_INPUTS,
    ignore_index: int = IGNORE_INDEX,
) -> None:
    with open(checkpoint_dir / "lit_config.json", "r") as file:
        config = json.load(file)
        max_seq_length = config["block_size"]

    destination_path.mkdir(parents=True, exist_ok=True)
    data_file_path = destination_path / "processed_data.json"
    print("Loading data file...")
    with open(data_file_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    print("Loading tokenizer...")

    tokenizer = Tokenizer(checkpoint_dir)
    train_set, test_set = random_split(
        data, [1.0 - test_split_fraction, test_split_fraction], generator=torch.Generator().manual_seed(seed)
    )
    train_set, test_set = list(train_set), list(test_set)

    print(f"train has {len(train_set):,} samples")
    print(f"test has {len(test_set):,} samples")

    print("Processing train split ...")
    train_set = [
        prepare_sample(
            example=sample,
            tokenizer=tokenizer,
            max_length=max_seq_length,
            mask_inputs=mask_inputs,
            ignore_index=ignore_index,
        )
        for sample in tqdm(train_set)
    ]
    print(f"test set is {train_set[0]} ")

    torch.save(train_set, destination_path / "train.pt")

    print("Processing test split ...")
    test_set = [
        prepare_sample(
            example=sample,
            tokenizer=tokenizer,
            max_length=max_seq_length,
            mask_inputs=mask_inputs,
            ignore_index=ignore_index,
        )
        for sample in tqdm(test_set)
    ]
    torch.save(test_set, destination_path / "test.pt")


def download_if_missing(file_path: Path, file_url: str):
    if file_path.exists():
        return
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(requests.get(file_url).text)


def prepare_sample(
    example: dict,
    tokenizer: Tokenizer,
    max_length: int,
    mask_inputs: bool = MASK_INPUTS,
    ignore_index: int = IGNORE_INDEX,
):
    full_prompt = generate_prompt(example)
    full_prompt_and_response = full_prompt + example["output"]
    encoded_full_prompt = tokenizer.encode(full_prompt, max_length=max_length)
    encoded_full_prompt_and_response = tokenizer.encode(full_prompt_and_response, eos=True, max_length=max_length)

    # The labels are the full prompt with response, but with the prompt masked out
    labels = encoded_full_prompt_and_response.clone()
    if mask_inputs:
        labels[: len(encoded_full_prompt)] = ignore_index

    return {
        **example,
        "input_ids": encoded_full_prompt_and_response,
        "input_ids_no_response": encoded_full_prompt,
        "labels": labels,
    }


def generate_prompt(example):

    if example["input"]:
        return (
            "Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:"
        )
    return (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        f"### Instruction:\n{example['instruction']}\n\n### Response:"
    )


In [ ]:
prepare()

In [14]:
!python finetune/adapter.py \
  --data_dir /content/drive/MyDrive \
  --checkpoint_dir checkpoints/tiiuae/falcon-7b \
  --out_dir /content/drive/MyDrive/counsel_chat \
  --precision bf16-true

/content/lit-gpt/finetune/adapter.py:297: JsonargparseDeprecationWarning: 
    Only use the public API as described in https://jsonargparse.readthedocs.io/en/stable/#api-reference.
    Importing from jsonargparse.cli is kept only to avoid breaking code that does not correctly use the public
    API. It will no longer be available from v5.0.0.

  from jsonargparse.cli import CLI
{'eval_interval': 600, 'save_interval': 250, 'eval_iters': 100, 'log_interval': 10, 'devices': 1, 'learning_rate': 0.003, 'batch_size': 64.0, 'micro_batch_size': 2, 'gradient_accumulation_iters': 32.0, 'epoch_size': 50000, 'num_epochs': 1, 'max_iters': 25000, 'weight_decay': 0.02, 'warmup_steps': 1562.0}
Global seed set to 1337
Loading model 'checkpoints/tiiuae/falcon-7b/lit_model.pth' with {'org': 'tiiuae', 'name': 'falcon-7b', 'block_size': 2048, 'vocab_size': 50254, 'padding_multiple': 512, 'padded_vocab_size': 65024, 'n_layer': 32, 'n_head': 71, 'n_embd': 4544, 'rotary_percentage': 1.0, 'parallel_residual': 

In [ ]:
!python generate/adapter.py \
  --adapter_path /content/drive/MyDrive/counsel_chat/iter-007999-ckpt.pth \
  --checkpoint_dir checkpoints/tiiuae/falcon-7b \
  --prompt "I feel depressed and I don't know what to do."